# Tutorial #4: Provision a feature store in various ways

Azure ML managed feature store lets you discover, create and operationalize features. Features are the connective tissue in ML lifecycle, starting from prototyping where you experiment with various features to operationalization where models are deployed and feature data is looked up during inference. For information on basics concept of feature store, see [feature store concepts](https://learn.microsoft.com/azure/machine-learning/concept-what-is-managed-feature-store).

In this tutorial series you will experience various approach to provision a feature store:

This tutorial is an enrichment for the Notebook 1 . In this tutorial you will:
- Create a minimal feature store resource via CLI
- Create a feature store with specifying resources via CLI
- Update a feature store with specifying resources via CLI
- Create a minimal feature store resource via SDK
- Create a feature store with specifying resources via SDK
- Update a feature store with specifying resources via SDK


## Prerequisites and Setup

Prerequisites and Setup is the same as the prerequisites and Setup part in `Tutorial #1: Develop a feature set and register with managed feature store`

#### Start spark session
Execute the following code cell to start the Spark session. It wil take approximately 10 minutes to install all dependencies and start the Spark session.

In [ ]:
# Run this cell to start the spark session (any code block will start the session). This can take around 10 mins.
print("start spark session")

#### Setup root directory for the samples
This code cell sets up the root directory for the samples.

In [ ]:
import os

# Please update your alias belpw (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure in the left nav
root_dir = "./Users/<your_user_alias>/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

#### Setup CLI

1. Install AzureML CLI extention
1. Authenticate
1. Set the default subscription

In [ ]:
!az extension add --name ml

In [ ]:
# Authenticate
!az login

In [ ]:
# Set default subscription
import os

subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]

!az account set -s $subscription_id

## Note
Feature store Vs Project workspace: You will use a feature store to reuse features across projects. You will use a project workspace (the current workspace) to train and inference models, by leveraging features from feature stores. Many project workspaces can share and reuse a same feature store.

## Note
In this tutorial you will be using CLI and feature store CRUD SDK:

1. CLI: You will use CLI for CRUD (create, read, update, and delete) operations on feature store, feature set and feature store entities.
2. Feature store CRUD SDK:    You will use the same SDK, MLClient (package name `azure-ai-ml`), that you use with Azure ML workspace. This will be used for feature store CRUD operations (create, read, update, and delete) for feature store, feature set and feature store entities. This is because feature store is implemented as a type of workspace. 

## Option 1: Provision a feature store via CLI

In [ ]:
# We use the subscription, resource group, region of this active project workspace.
# You can optionally replace them to create the resources in a different subsciprtion/resourceGroup, or use existing resources
import os

featurestore_name = "FEATURESTORE-NAME"
featurestore_location = "eastus"
featurestore_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
featurestore_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]

### 1.1 Default feature store provision experience

In the command, you only need to provide the subscription, resource group, feature store name and the location, the system will help you to provision a minimal feature store.

In this default feature store provision experience, the system will:

(1) provision a storage account with storage type as `Azure Data Lake Storage Gen2` that used as workspace default storage account

(2) create a container in the above storage as offline store.

(3) provision an User managed identity(UAI).

(4) provision the RBAC of UAI to workspace and to offline store.

    Below is the details RBAC permission the system helps to grant:

|Scope|	Action/Role|
|--|--|
|Feature store | `AzureML Data Scientist` role|
|Container as feature store offline store | `Storage Blob Data Contributor` role|
|Storage account as workspace default storage account	|`Storage Blob Data Contributor` role|

(5) create an offline store connection to the feature store workspace

(6) provision other resources for workspace including keyvault etc.

In [ ]:
!az ml feature-store create --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --location $featurestore_location --name $featurestore_name

### 1.2 Create a feature store with specifying resources
Users has the flexibility to bring resources instead of using the system default provisioning ones.

Resources that user could specify are:

1. materialization_identity(UAI): Optional. 

    a. if user brings nothing, then the system will help to provision one.

    b. if user brings a UAI. the system will help to provision feature store with that UAI, if user does not skip grant permission in CLI/SDK (by defult, it's False), the the system will help to grant proper permision to the UAI for dependent resources. if user explicitly ask for skipping grant permission, then user could follow the steps in `Grant RBAC permission to the user assigned managed identity (UAI)` section in this notebook to grant permissions manually.


2. storage_account: Optional. 

    This will be used as the default storage for the feature store workspace. It could be either blob storage or `Azure Data Lake Storage Gen2` storage.

    a. If user brings nothing, then the system will help to provision an `Azure Data Lake Storage Gen2` storage account, this will be used as the default storage for the feature store worksapce. Furthermore, if user also brings nothing for offline store, then the system will create a container in this storage account, and take this container as offline store.

    b. If user brings a default blob storage account, then system will use it as the default storage for the feature store worksapce. Furthermore, if user brings nothing for offline store, then the system will provision a seperate `Azure Data Late Storage Gen2` storage account and create a container in it as offline store.

    c. If user brings a `Azure Data Lake Storage Gen2` storage account, then system will use it as the default storage for the feature store worksapce. Furthermore, if user brings nothing for offline store, then the system will create a container in it, and take this container as offline store.


3. offline store: Optional.

    a. If user brings nothing, then the system will help to provision a `Azure Data Lake Storage Gen2` storage account, and then create a container in it as offline store.

    b. If user brings a container as offline store, then the system will use it as the offline store. in the meantime, if user does not bring a storage account, then the system will provision an `Azure Data Lake Storage Gen2` storage account as the default blob storage for the workspace.


4. online store: Optional.

    a. If user brings nothing, then the feature store will be provisioned without online store. user could update the feature store later if need to add the online store.

    b. If user brings an `Azure Cache for Redis`, then the system will create feature store with online store connection targeting to the user's `Azure Cache for Redis`.
        

** During storage account creation in Aure portal, if with the `Hierarchical namespace` enabled, it will be a Azure Data Lake Storage Gen2 storage account, if with the `Hierarchical namespace` disabled, then it will be blob storage account.


We provide multiple options for user to specify resources for feature store.

It includes:

1. --parameter: Using params in the CLI to specify resources. user could find all supported params here: https://learn.microsoft.com/en-us/cli/azure/ml/feature-store?view=azure-cli-latest#az-ml-feature-store-create

2. --file: Using a yml file to specify resources.

3. --set: Using --set in the CLI to specify resources.

** Note --set and --yaml is mutual exclusive. users are not allowed to use them in the same CLI.

In this section, there will be examples on how to use them.

#### 1.2.0 Prepare your resources

Resources that user could specify during feature store provisioning are: User assigned identity (UAI), offline store, online store, storage account

For online store, it will be set only when it's specified.

For UAI, offline store and storage account, if they are not specified by user, then the system will help to provision one.

##### Create your resources via Azure Portal UI
All resources could be created via [Azure Portal UI](https://ms.portal.azure.com/#create/hub), the steps are search service name, and then follow the directions to create the resource, and go to the resource -> Overview -> click the "JSON View" on the right-above corner -> find the Resource Id, it's needed during specifying the resources when provisioning the feature store.

Below is the service name/description for each resource:
* UAI --- User Assigned Managed Identity
* Offline store -- A container in `Azure Data Lake Storage Gen2` storage account
* Online store -- Azure Cache for Redis
* Storage account -- Storage used as the workspace default storage


##### Create your resources via CLI
If you don't want to use the portal, then you could run the following cells to create resources

In [ ]:
import os
from azure.ai.ml import MLClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

project_ws_sub_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
project_ws_rg = os.environ["AZUREML_ARM_RESOURCEGROUP"]
project_ws_name = os.environ["AZUREML_ARM_WORKSPACE_NAME"]

# Connect to the project workspace
ws_client = MLClient(
    AzureMLOnBehalfOfCredential(), project_ws_sub_id, project_ws_rg, project_ws_name
)

##### Create a container in `Azure Data Lake Gen2` storage account that will be used as offline store.
Provide the name of an Azure Data Lake Storage Gen2 storage account in the following code sample. Other than that, you can execute the following code cell with the provided default settings. Optionally, you can override the default settings.

In [ ]:
## Default Setting
# We use the subscription, resource group, region of this active project workspace,
# We hard-coded default resource names for creating new resources

## Overwrite
# You can replace them if you want to create the resources in a different subsciprtion/resourceGroup, or use existing resources
# At the minimum, provide an ADLS Gen2 storage account name for `storage_account_name`

storage_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
storage_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]
offline_store_storage_account_name = "OFFLINE-STORE-ACCOUNT-NAME"

storage_location = "eastus"
offline_store_storage_file_system_name = "OFFLINE-STORE-CONTAINER-NAME"

Execute the following code cell to create the ADLS Gen2 storage account defined in the above code cell.

In [ ]:
# Create new storage account
!az storage account create --name $offline_store_storage_account_name --enable-hierarchical-namespace true --resource-group $storage_resource_group_name --location $storage_location --subscription $storage_subscription_id

Execute the following code cell to create a new storage container for offline store.

In [ ]:
# Create a new storage container for offline store
!az storage fs create --name $offline_store_storage_file_system_name --account-name $offline_store_storage_account_name --subscription $storage_subscription_id

##### Create storage account that will be used to as workspace default storage account

In [ ]:
## Default Setting
# We use the subscription, resource group, region of this active project workspace,
# We hard-coded default resource names for creating new resources

## Overwrite
# You can replace them if you want to create the resources in a different subsciprtion/resourceGroup, or use existing resources
# At the minimum, provide an ADLS Gen2 storage account name for `storage_account_name`

storage_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
storage_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]
storage_account_name = "STORAGE-ACCOUNT-NAME"

storage_location = "eastus"

In [ ]:
# Create new storage account
!az storage account create --name $storage_account_name --enable-hierarchical-namespace true --resource-group $storage_resource_group_name --location $storage_location --subscription $storage_subscription_id

##### Create a new Redis instance as online store

Online store will be provisioned for the feature store only when it's been specified during creation/update.

User could go to the [Azure portal UI](https://ms.portal.azure.com/#create/hub) --> search for "Azure Cache for Redis", and click the "Azure Cache for Redis" app, then follow the directions to create one.

Or follow the following steps to create one

In the following code cell, define the name of the Azure Cache for Redis that you want to create or reuse. Optionally, you can override other default settings.

In [ ]:
ws_location = ws_client.workspaces.get(ws_client.workspace_name).location

redis_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
redis_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]
redis_name = "REDIS-NAME"
redis_location = ws_location
sku = "premium"
size = "P2"

You can select the Redis cache tier (basic, standard, premium, or enterprise). You should choose a SKU family that is available for the selected cache tier. See this documentation page to learn more about [how selecting different tiers may affect cache performance](https://learn.microsoft.com/azure/azure-cache-for-redis/cache-best-practices-performance). See this link learn more about [pricing for different SKU tiers and families of Azure Cache for Redis](https://azure.microsoft.com/en-us/pricing/details/cache/).

Execute the following code cell to create an Azure Cache for Redis with premium tier, SKU family P and cache capacity 2. It may take approximately 5-10 minutes to provision the Redis instance.

In [ ]:
!az redis create --name $redis_name --resource-group $redis_resource_group_name --location "$redis_location" --sku $sku --vm-size $size

##### Setup user-assigned managed identity (UAI)
In the following code cell, provide a name for the user-assigned managed identity that you would like to create.

In [ ]:
# User assigned managed identity values. Optionally you may change the values.
uai_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
uai_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]
uai_name = "UAI-NAME"
# feature store location is used by default. You can change it.
uai_location = storage_location

Execute the following cell to create the UAI.

In [ ]:
!az identity create --subscription $uai_subscription_id --resource-group $uai_resource_group_name --location $uai_location --name $uai_name

##### Retrieve UAI properties
The following code retrieves principal ID, client ID, and ARM ID property values for the created UAI.

In [ ]:
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

msi_client = ManagedServiceIdentityClient(
    AzureMLOnBehalfOfCredential(), uai_subscription_id
)
managed_identity = msi_client.user_assigned_identities.get(
    resource_name=uai_name, resource_group_name=uai_resource_group_name
)

uai_principal_id = managed_identity.principal_id
uai_client_id = managed_identity.client_id
uai_arm_id = managed_identity.id

##### Grant RBAC permission to the user assigned managed identity (UAI)

User needs to grant RBAC permission manually only when user ask the system not to grant permission (use --not-grant-permissions True in CLI command or use the grant_materialization_permissions=False in SDK). 

If user uses --not-grant-permissions True in CLI command or use the grant_materialization_permissions=False in SDK during feature store creation, then user need to grant the UAI with the following permission:

|Scope|	Action/Role|
|--|--|
|Feature store | `AzureML Data Scientist` role|
|Container as feature store offline store | `Storage Blob Data Contributor` role|
|Storage account as workspace default storage account	|`Storage Blob Data Contributor` role|
|Azure Cache for Redis as online store	|`Contributor` role|

The below cli commands will assign **Storage Blob Data Contributor** role to the UAI. To learn more about access control, see role-based access control for [Azure storage accounts](https://learn.microsoft.com/azure/storage/blobs/data-lake-storage-access-control-model#role-based-access-control-azure-rbac) and [Azure Machine Learning workspace](https://learn.microsoft.com/azure/machine-learning/how-to-assign-roles).

In [ ]:
# feature store arm id
feature_store_arm_id = "/subscriptions/{sub_id}/resourceGroups/{rg}/providers/Microsoft.MachineLearningServices/workspaces/{ws_name}".format(
    sub_id=featurestore_subscription_id,
    rg=featurestore_resource_group_name,
    ws_name=featurestore_name,
)
print(feature_store_arm_id)

# set the container arm id
offline_store_gen2_container_arm_id = "/subscriptions/{sub_id}/resourceGroups/{rg}/providers/Microsoft.Storage/storageAccounts/{account}/blobServices/default/containers/{container}".format(
    sub_id=storage_subscription_id,
    rg=storage_resource_group_name,
    account=offline_store_storage_account_name,
    container=storage_file_system_name_offline_store,
)

print(offline_store_gen2_container_arm_id)

storage_account_gen2_container_arm_id = "/subscriptions/{sub_id}/resourceGroups/{rg}/providers/Microsoft.Storage/storageAccounts/{account}".format(
    sub_id=storage_subscription_id,
    rg=storage_resource_group_name,
    account=storage_account_name,
)

print(storage_account_gen2_container_arm_id)

redis_arm_id = "/subscriptions/{sub_id}/resourceGroups/{rg}/providers/Microsoft.Cache/Redis/{redis_name}".format(
    sub_id=storage_subscription_id,
    rg=storage_resource_group_name,
    redis_name=redis_name,
)

print(redis_arm_id)

In [ ]:
!az role assignment create --role "AzureML Data Scientist" --assignee-object-id $uai_principal_id --assignee-principal-type ServicePrincipal --scope $feature_store_arm_id

In [ ]:
!az role assignment create --role "Storage Blob Data Contributor" --assignee-object-id $uai_principal_id --assignee-principal-type ServicePrincipal --scope $offline_store_gen2_container_arm_id

In [ ]:
!az role assignment create --role "Storage Blob Data Contributor" --assignee-object-id $uai_principal_id --assignee-principal-type ServicePrincipal --scope $storage_account_gen2_container_arm_id

In [ ]:
!az role assignment create --role "Contributor" --assignee-object-id $uai_principal_id --assignee-principal-type ServicePrincipal --scope $redis_arm_id

#### 1.2.1 Specify resources via yml file
After the resources are ready, modify the feature-store.yaml file to specify resources.

** User could remove the section if don't want to specify one, then the system will provision a default one.

In [ ]:
# Specify the resources you want to the use for your feature store, for resources that not specified in the yml file, the service will help to provision one.
yml_file = root_dir + "/featurestore/feature-store-specify-resources.yml"

!cat $yml_file

In [ ]:
!az ml feature-store create --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --location $featurestore_location --name $featurestore_name --file $yml_file

#### 1.2.2 Specify resources via params

All supported params could be found here: https://learn.microsoft.com/en-us/cli/azure/ml/feature-store?view=azure-cli-latest#az-ml-feature-store-create

In [ ]:
import json

# The value of the param should be a json string with double quote. a safe way to contruct the json string is to contruct one, and then use the json.dumps to get one, so that no errors on the format.
# Here take the offline store as an example, the format will be '{"target": "<resource_id>","type":"<store_type>"}'
# Remember to replace <sub_id>, <rg>, <offline-store-account-name> and <offline-store-container-name> to the right value.
json_string = '{"target":"/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<offline-store-account-name>/blobServices/default/containers/<offline-store-container-name>","type":"azure_data_lake_gen2"}'

offline_store_json = json.dumps(json_string)

print(offline_store_json)

In [ ]:
!az ml feature-store create --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --location $featurestore_location --name $featurestore_name --offline-store $offline_store_json

#### 1.2.3 Specifying resources via set option

The format to set the resources via --set is propertyA.property1=<value> propertyA.property2=<value>, use a space to split values for each properties.

In [ ]:
# Take offline store as an example to set the value. there are two properties for offline store, target and type. so need to set them explicitly in the CLI
# Remember to replace <sub_id>, <rg>, <offline-store-account-name> and <offline-store-container-name> to the right value.
offline_store_target = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<offline-store-account-name>/blobServices/default/containers/<offline-store-container-name>"
offline_store_type = "azure_data_lake_gen2"

In [ ]:
# when using --set, need to use a space to split different values
!az ml feature-store create --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --location $featurestore_location --name $featurestore_name --set offline_store.type=$offline_store_type offline_store.target=$offline_store_target

#### 1.2.4 Specifying resources in a mixed usage
User could use mixed ways to specify resources.

For edge case that one resource is specified in different ways with different values. Take offline store as an example, using --offline-store, --set and --file to set the offline store with different values, the priority will be (--offline-store) > (--set) and (--file), that is, the system will take the value specified by --offline-store as the value, and ignore the values in --set or --file.

**Note:
--set and --file is mutual exclusive, user should not use them in the same time.

In [ ]:
import json

# Remember to replace <sub_id>, <rg>, <offline-store-account-name> and <offline-store-container-name> to the right value.
json_string = '{"target":"/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<offline-store-account-name>/blobServices/default/containers/<offline-store-container-name>","type":"azure_data_lake_gen2"}'

offline_store_json = json.dumps(json_string)

# materialization_identity
client_id = "<your-uai-client-id>"
resource_id = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.ManagedIdentity/userAssignedIdentities/<your-uai-name>"

# Specify the resources you want to the use for your feature store, for resources that not specified in the yml file, the service will help to provision one.
yml_file = root_dir + "/featurestore/feature-store-specify-resources.yml"

!cat $yml_file

In [ ]:
!az ml feature-store create --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --location $featurestore_location --name $featurestore_name --offline-store $offline_store_json --file $yml_file

In [ ]:
!az ml feature-store create --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --location $featurestore_location --name $featurestore_name --offline-store $offline_store_json --set materialization_identity.client_id=$client_id materialization_identity.resource_id=$resource_id

### 1.3 Update feature store via CLI

https://learn.microsoft.com/en-us/cli/azure/ml/feature-store?view=azure-cli-latest#az-ml-feature-store-update

With using file, user is allowed to update offline-store, UAI, online store

In [ ]:
featurestore_name = "FEATURESTORE-NAME"
yml_file = root_dir + "/featurestore/feature-store-update.yml"

!cat $yml_file

In [ ]:
!az ml feature-store update --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --name $featurestore_name --file $yml_file

## Option 2. Create feature store using SDK

In [ ]:
# We use the subscription, resource group, region of this active project workspace.
# You can optionally replace them to create the resources in a different subsciprtion/resource group, or use existing resources.
import os

featurestore_name = "FEATURESTORE-NAME"
featurestore_location = "eastus"
featurestore_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
featurestore_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]

### 2.1 Default feature store provision experience

In the cell, you only need to provide the subscription, resource group, feature store name and the location via SDK, the system will help you to provision a minimal feature store.

The default feature store provision experience could see the `1.1 Default feature store provision experience` section in this notebook.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import FeatureStore
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

ml_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
)

fs = FeatureStore(
    name=featurestore_name,
    location=featurestore_location,
)
# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

### 2.2 Create a feature store with specifying resources
If need to create resources, such as offline store, online store, storage account or UAI, please refer to 1.2.0 section for resource preparation.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    FeatureStore,
    MaterializationStore,
    ManagedIdentityConfiguration,
)
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

ml_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
)

# Remember to replace <sub_id>, <rg>, <offline-store-account-name> and <offline-store-container-name> to the right value.
# Specify offline store
gen2_container_arm_id = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<offline-store-account-name>/blobServices/default/containers/<offline-store-container-name>"
offline_store = MaterializationStore(
    type="azure_data_lake_gen2", target=gen2_container_arm_id
)

# Specify UAI
materialization_identity = ManagedIdentityConfiguration(
    client_id="<uai-client-id>",
    principal_id="uai-principal-id",
    resource_id="/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.ManagedIdentity/userAssignedIdentities/<uai-name>",
)

# Specify storage account
storage_account = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<storage-account-name>"

# Specify online store
redis_arm_id = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Cache/Redis/<redis-acount-name>"
online_store = MaterializationStore(type="redis", target=redis_arm_id)

# Add them into the FeatureStore if need to specify the resources, name and location are required.
# offline_store, materialization_identity, storage_account are optional, if user does not specify them, then the system will provision one by default
# online_store is optional, if user does not specify it, then the feature store will be provisioned without online_store.
fs = FeatureStore(
    name=featurestore_name,
    location=featurestore_location,
    offline_store=offline_store,
    materialization_identity=materialization_identity,
    storage_account=storage_account,
    online_store=online_store,
)
# wait for feature store creation
fs_poller = ml_client.feature_stores.begin_create(fs)
print(fs_poller.result())

### 2.3 Update resources for feature store

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    FeatureStore,
    MaterializationStore,
    ManagedIdentityConfiguration,
)
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

ml_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
)

# Remember to replace <sub_id>, <rg>, <offline-store-account-name> and <offline-store-container-name> to the right value.
# Specify offline store
gen2_container_arm_id = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<offline-store-account-name>/blobServices/default/containers/<offline-store-container-name>"
offline_store = MaterializationStore(
    type="azure_data_lake_gen2", target=gen2_container_arm_id
)

# Specify UAI
materialization_identity = ManagedIdentityConfiguration(
    client_id="<uai-client-id>",
    principal_id="<uai-principal-id>",
    resource_id="/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.ManagedIdentity/userAssignedIdentities/<uai-name>",
)

# Specify storage account
storage_account = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Storage/storageAccounts/<storage-account-name>"

# Specify online store
redis_arm_id = "/subscriptions/<sub_id>/resourceGroups/<rg>/providers/Microsoft.Cache/Redis/<redis-account-name>"
online_store = MaterializationStore(type="redis", target=redis_arm_id)

# By adding the specified resources to FeatureStore, the feature store will be updated to use the resources specified here.
# User is allowed to update resources like: offline_store, materialization_identity and online_store.
fs = FeatureStore(
    name=featurestore_name,
    offline_store=offline_store,
    materialization_identity=materialization_identity,
    online_store=online_store,
)
# wait for feature store update.
# If need to grant materialization permission, then set the value to True,
#    for resources that user has already assigned the role assignment,
#    then could set the grant_materialization_permissions=False to skip the role assignments.
fs_poller = ml_client.feature_stores.begin_update(
    fs, grant_materialization_permissions=True
)
print(fs_poller.result())

#### Explore the feature store UI
* Goto the [Azure ML global landing page](https://ml.azure.com/home).
* Click on **Feature stores** in the left navigation.
* You will see the list of feature stores that you have access to. Click on the feature store that you have created above.

You can see the feature sets and entities that you have created.

Note: Creating and updating feature store assets (feature sets and entities) is possible only through SDK and CLI. You can use the UI to search/browse the feature store.

## Clean up

#### Delete feature store via CLI

Delete all resources together with the feature store by specifying --all-resources.
This will delete all resources for the feature store except offline store, online store and UAI.

In [ ]:
!az ml feature-store delete --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --name $featurestore_name --all-resources -y

Delete feature store only while keep all other provisioned resources

In [ ]:
!az ml feature-store delete --subscription $featurestore_subscription_id --resource-group $featurestore_resource_group_name --name $featurestore_name -y

#### Delete feature store via SDK
If need to delete dependent resources, then need to set delete_dependent_resources=True, if not set the value, then it will be False by default.

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

ml_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
)

# wait for feature store creation.
# set delete_dependent_resources=True if need to delete dependent resources
# set delete_dependent_resources=False to skip the dependent resources deletion
fs_poller = ml_client.feature_stores.begin_delete(
    name="<feature store name>", delete_dependent_resources=True
)
print(fs_poller.result())

## Next steps
* Develop a feature set and register with managed feature store